# Citywide Cellular Traffic Prediction Based on Densely Connected Convolutional Neural Networks

In [81]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #空间三维画图
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from collections import OrderedDict

### Dataset Input

## Model Structure

In [72]:
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features))
        self.add_module('relu1', nn.ReLU(inplace=True))
        self.add_module('conv1', nn.Conv2d(
            num_input_features, bn_size * growth_rate,
            kernel_size=1, stride=1, bias=False
        ))
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate))
        self.add_module('relu2', nn.ReLU(inplace=True))
        self.add_module('conv2', nn.Conv2d(
            bn_size * growth_rate, growth_rate,
            kernel_size=3, stride=1, padding=1, bias=False
        ))
        self.drop_rate = drop_rate

    def forward(self, input):
        new_features = super(_DenseLayer, self).forward(input)
        if self.drop_rate > 0:
            new_features = F.dropout(
                new_features, p=self.drop_rate,
                training=self.training
            )
        return torch.cat([input, new_features], 1)

In [73]:
class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super().__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate,bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)

In [74]:
class DenseNetUnit(nn.Sequential):
    def __init__(
        self, channels, nb_flows, layers=5, growth_rate=12,
        num_init_features=32, bn_size=4, drop_rate=0.2
    ):
        super().__init__()

        if channels > 0:
            self.features = nn.Sequential(OrderedDict([
                ('conv0', nn.Conv2d(channels, num_init_features, kernel_size=3, padding=1)),
                ('norm0', nn.BatchNorm2d(num_init_features)),
                ('relu0', nn.ReLU(inplace=True))
            ]))

            # Dense Block
            num_features = num_init_features
            num_layers = layers
            block = _DenseBlock(
                num_layers=num_layers, num_input_features=num_features,
                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate
            )
            self.features.add_module('denseblock', block)
            num_features = num_features + num_layers * growth_rate

            # Final batch norm
            self.features.add_module('normlast', nn.BatchNorm2d(num_features))
            self.features.add_module('convlast', nn.Conv2d(
                num_features, nb_flows,
                kernel_size=1, padding=0, bias=False
            ))

            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight.data)
                elif isinstance(m, nn.BatchNorm2d):
                    m.weight.data.fill_(1)
                    m.bias.data.zero_()
                elif isinstance(m, nn.Linear):
                    m.bias.data.zero_()
        else:
            pass

    def forward(self, x):
        out = self.features(x)
        return out

In [75]:
class DenseNet(nn.Module):
    def __init__(self, nb_flows, drop_rate, channels):
        super().__init__()
        self.drop_rate = drop_rate
        self.nb_flows = nb_flows
        self.channels = channels

        self.close_feature  = DenseNetUnit(channels[0], nb_flows)
        self.period_feature = DenseNetUnit(channels[1], nb_flows)
        self.trend_feature  = DenseNetUnit(channels[2], nb_flows)

    def forward(self, inputs):
        out = 0
        if self.channels[0] > 0:
            out += self.close_feature(inputs[0])
        if self.channels[1] > 0:
            out += self.period_feature(inputs[1])
        if self.channels[2] > 0:
            out += self.trend_feature(inputs[2])

        return torch.sigmoid(out)

## Model Train & Test

In [87]:
def set_lr(optimizer, epoch, n_epochs, lr):
    lr = lr
    if float(epoch) / n_epochs > 0.75:
        lr = lr * 0.01
    if float(epoch) / n_epochs > 0.5:
        lr = lr * 0.1

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

Train

In [93]:
def train_epoch(model, data_type='train', period_size, close_size, trend_size):
    total_loss = 0
    
    if data_type == 'train':
        model.train()
        data = train_loader
        
    elif data_type == 'valid':
        model.eval()
        data = valid_loader

    if (period_size > 0) & (close_size > 0) & (trend_size > 0):
        for idx, (c, p, t, target) in enumerate(data):
            optimizer.zero_grad()
            model.zero_grad()
            input_var = [Variable(_.float()).cuda(async=True) for _ in [c, p, t]]
            target_var = Variable(target.float(), requires_grad=False).cuda(async=True)

            pred = model(input_var)
            loss = criterion(pred, target_var)
            total_loss += loss.data[0]
            loss.backward()
            optimizer.step()
            
    elif (close_size > 0) & (period_size > 0):
        for idx, (c, p, target) in enumerate(data):
            optimizer.zero_grad()
            model.zero_grad()
            input_var = [Variable(_.float()).cuda(async=True) for _ in [c, p]]
            target_var = Variable(target.float(), requires_grad=False).cuda(async=True)

            pred = model(input_var)
            loss = criterion(pred, target_var)
            total_loss += loss.data[0]
            loss.backward()
            optimizer.step()
            
    elif close_size > 0:
        for idx, (c, target) in enumerate(data):
            optimizer.zero_grad()
            model.zero_grad()
            x = [Variable(c.float()).cuda(async=True)]
            y = Variable(target.float(), requires_grad=False).cuda(async=True)

            pred = model(x)
            loss = criterion(pred, y)
            total_loss += loss.data[0]
            loss.backward()
            optimizer.step()

    return total_loss


def train(learningRate, optimizer, epochSize):
    best_valid_loss = 1.0
    train_loss, valid_loss = [], []
    
    for i in range(epochSize):
        learningRate = set_lr(optimizer, i, epochSize, learningRate)
        train_loss.append(train_epoch('train'))
        valid_loss.append(train_epoch('valid'))

        if valid_loss[-1] < best_valid_loss:
            best_valid_loss = valid_loss[-1]

            torch.save({'epoch': i, 'model': model, 'train_loss': train_loss,
                        'valid_loss': valid_loss}, model_filename + '.model')
            torch.save(optimizer, model_filename + '.optim')

        print((
            'iter: [{:d}/{:d}], train_loss: {:0.6f}, valid_loss: {:0.6f}, '
            'best_valid_loss: {:0.6f}, lr: {:0.5f}'
        ).format(
            (i + 1), epochSize,
            train_loss[-1], valid_loss[-1],
            best_valid_loss, learningRate
        ))

Test

In [95]:
def predict(test_type='train'):
    predictions = []
    ground_truth = []
    loss = []
    best_model = torch.load('../data/best.model').get('model')

    if test_type == 'train':
        data = train_loader
    elif test_type == 'test':
        data = test_loader
    elif test_type == 'valid':
        data = valid_loader

    if (sizePeriod > 0) & (sizeClose > 0) & (sizeTrend > 0):
        for idx, (c, p, t, target) in enumerate(data):
            input_var = [Variable(_.float()).cuda(async=True) for _ in [c, p, t]]
            target_var = Variable(target.float(), requires_grad=False).cuda(async=True)
            pred = best_model(input_var)
            predictions.append(pred.data.cpu().numpy())
            ground_truth.append(target.numpy())
            loss.append(criterion(pred, target_var).data[0])
            
    elif (sizeClose > 0) & (sizePeriod > 0):
        for idx, (c, p, target) in enumerate(data):
            input_var = [Variable(_.float()).cuda(async=True) for _ in [c, p]]
            target_var = Variable(target.float(), requires_grad=False).cuda(async=True)
            pred = best_model(input_var)
            predictions.append(pred.data.cpu().numpy())
            ground_truth.append(target.numpy())
            loss.append(criterion(pred, target_var).data[0])
            
    elif sizeClose > 0:
        for idx, (c, target) in enumerate(data):
            input_var = Variable(c.float()).cuda(async=True)
            target_var = Variable(target.float(), requires_grad=False).cuda(async=True)
            pred = best_model(input_var)
            predictions.append(pred.data.cpu().numpy())
            ground_truth.append(target.numpy())
            loss.append(criterion(pred, target_var).data[0])

    final_predict = np.concatenate(predictions)
    ground_truth = np.concatenate(ground_truth)

    rmse = []
    for y_hat, y in zip(final_predict, ground_truth):
        flows, height, width = y_hat.shape
        y_hat = np.reshape(y_hat, (flows, height * width)) * (mmn.max - mmn.min)
        y = np.reshape(y, (flows, height * width)) * (mmn.max - mmn.min)
        rmse.append(metrics.mean_squared_error(y_hat, y) ** 0.5)

    if test_row & test_col:
        row, col = test_row, test_col
    else:
        row_length, col_length = ground_truth.shape[-2:]
        row, col = int(row_length/2), int(col_length/2)

SyntaxError: invalid syntax (Temp/ipykernel_15968/3933513678.py, line 16)

## Simulation

Parameters

In [78]:
sizeClose  = 3
sizePeriod = 0
sizeTrend  = 0
learningRate = 0.001
batchSize = 32
epochSize = 300
dropRate = 0.0

loss = 'l2'
nb_flow = 2

In [94]:
# get data channels
channels = [sizeClose  * nb_flow,
            sizePeriod * nb_flow,
            sizeTrend  * nb_flow]

model = DenseNet(
    nb_flows = nb_flow, drop_rate = dropRate, channels = channels
).cuda()
optimizer = optim.Adam(model.parameters(), learningRate)

if loss == 'l1':
    criterion = nn.L1Loss().cuda()
elif loss == 'l2':
    criterion = nn.MSELoss().cuda()

train()

UnboundLocalError: local variable 'learningRate' referenced before assignment

In [ ]:
predict('test')

plt.figure()
plt.plot(torch.load('../data/best.model').get('train_loss')[1:-1], 'r-')
plt.legend(labels=['train_loss'], loc='best')

plt.figure()
plt.plot(torch.load('../data/best.model').get('valid_loss')[:-1], 'k-')
plt.legend(labels=['test_loss'], loc='best')